In [1]:
import pandas as pd
import os
import re
from tslearn.metrics import dtw, dtw_path
from tslearn.barycenters import dtw_barycenter_averaging

df_train = pd.read_pickle(os.path.join(r"data/df_train.pkl"))

df_test = pd.read_pickle(os.path.join(r"data/df_test.pkl"))

df_train = pd.concat([df_train,df_test], ignore_index=True)
# print(df_train)
# print(df_test)

print(df_train["Débit horaire"])

0        746.0
1        549.0
2        503.0
3        500.0
4        525.0
         ...  
29004    649.0
29005    473.0
29006    283.0
29007    174.0
29008    138.0
Name: Débit horaire, Length: 29009, dtype: float64


In [2]:
mapper = {'AV_Champs_Elysees': 0, 'Convention': 1, 'Sts_Peres': 2}
df_train['Libelle'] = df_train['Libelle'].map(mapper)
df_test['Libelle'] = df_test['Libelle'].map(mapper)

mapper_2 = {'Invalide': "0", 'Barré': "1"}
df_train['Etat arc'] = df_train['Etat arc'].map(mapper_2)
df_test['Etat arc'] = df_test['Etat arc'].map(mapper_2)
print(df_train["Libelle"].unique())

print(df_train["precipMM"].unique())

[0 1 2]
[0.  1.2 1.6 0.8 2.7 3.6 1.8 1.3 0.9 0.1 0.2 0.7 1.  0.5 1.1 1.5 0.4 0.6
 0.3 2.1 1.4 1.7 2.  1.9 2.6 3.5 2.2 2.4 3.2 4.4 5.9 3.  2.8 4.2 2.9 2.5
 3.7 4.  3.8 2.3]


In [3]:
print(df_train.columns)
#cat_columns = ["Etat trafic", "Etat arc", 'Etat du confinement', 'Couvre-feu',
#       'Jour férié']
cat_columns = ["Etat trafic", "Etat arc", 'Jour de la semaine_0', 'Jour de la semaine_1', 'Jour de la semaine_2',
       'Jour de la semaine_3', 'Jour de la semaine_4', 'Jour de la semaine_5',
       'Jour de la semaine_6', 'Etat du confinement', 'Couvre-feu',
       'Jour férié', 'Vacances scolaires']
bool_columns = ['Couvre-feu',
       'Jour férié', 'Vacances scolaires']
object_columns = ["Etat arc"]
# df_meta_train = pd.DataFrame()
# #df_meta_train[cat_columns] = df_train[cat_columns].apply(lambda x: x.astype(pd.CategoricalDtype(categories=df_meta.index)).cat.codes)
# df_meta_train[cat_columns] = df_train[cat_columns]
# df_meta_train['Libelle'] = df_train["Libelle"].astype(str)
# df_meta_train.index = df_meta_train['Libelle']

# df_meta_test = pd.DataFrame()
# #df_meta_train[cat_columns] = df_train[cat_columns].apply(lambda x: x.astype(pd.CategoricalDtype(categories=df_meta.index)).cat.codes)
# df_meta_test[cat_columns] = df_test[cat_columns]
# df_meta_test['Libelle'] = df_test["Libelle"].astype(str)
# df_meta_test.index = df_meta_test['Libelle']

# print(df_meta_test.index)
# print(df_meta_train['Etat arc'].unique())


# cat_code = dict(enumerate(df_meta_train.index))
# cat_code_reverse = {v:k for k,v in cat_code.items()}

print(df_train.dtypes)

Index(['Libelle', 'Date et heure de comptage', 'Débit horaire',
       'Taux d'occupation', 'Etat trafic', 'Etat arc', 'filename', 'Date',
       'Jour de la semaine_0', 'Jour de la semaine_1', 'Jour de la semaine_2',
       'Jour de la semaine_3', 'Jour de la semaine_4', 'Jour de la semaine_5',
       'Jour de la semaine_6', 'Etat du confinement', 'Couvre-feu',
       'Jour férié', 'Vacances scolaires',
       'Date des prochaines vacances scolaires',
       'Temps avant les prochaines vacances scolaires', 'tempC',
       'windspeedKmph', 'winddirDegree', 'weatherCode', 'precipMM', 'humidity',
       'visibility', 'pressure', 'cloudcover', 'HeatIndexC', 'DewPointC',
       'WindChillC', 'WindGustKmph', 'FeelsLikeC', 'hourly_uvIndex',
       'maxtempC', 'mintempC', 'avgtempC', 'totalSnow_cm', 'sunHour',
       'daily_uvIndex', 'sunrise', 'sunset', 'moon_phase', 'moon_illumination',
       'Journée'],
      dtype='object')
Libelle                                                    int64

In [4]:
def chunks(l, size, step):
    """ Yield successive n-sized chunks from l.
    """
    l = l[:(len(l) // size) * size]
    return [x for x in [l[i : i + size] for i in range(0, len(l), step)] if len(x) == size]

In [5]:
print(df_train["Libelle"])
print(df_test["Libelle"].unique())
print(df_train['Journée'])

0        0
1        0
2        0
3        0
4        0
        ..
29004    2
29005    2
29006    2
29007    2
29008    2
Name: Libelle, Length: 29009, dtype: int64
[]
0        False
1        False
2        False
3        False
4         True
         ...  
29004    False
29005    False
29006    False
29007    False
29008    False
Name: Journée, Length: 29009, dtype: bool


In [6]:
#paramètres
N_input = 14*24
N_output = 7*24

p_test = 0
#meta_specs = {'cat_input_dim': (len(cat_columns), len(df_meta_train.index)),
#              'embed_dim': 8}
optimizer_parameters = {'lr': 0.001}
batch_size = 32

In [7]:
from sklearn.preprocessing import MinMaxScaler
#visibility 
metatemp_columns = cat_columns + ['windspeedKmph', 'weatherCode', 'precipMM', 'cloudcover', 'HeatIndexC', 'maxtempC', 'mintempC']
num_columns = ['windspeedKmph', 'weatherCode', 'precipMM', 'cloudcover', 'HeatIndexC', 'maxtempC', 'mintempC']
df_train[bool_columns] = df_train[bool_columns].apply(lambda x : x.astype(int))
#df_test[bool_columns] = df_test[bool_columns].apply(lambda x : x.astype(int))
df_train[object_columns] = df_train[object_columns].apply(lambda x : x.astype(int))

metatemp_data_train = df_train.sort_values(['Date et heure de comptage', 'Libelle']).groupby('Libelle')[metatemp_columns].apply(lambda x: chunks(x.values, size=N_input+N_output, step=1))
#metatemp_data_test = df_test.sort_values(['Date et heure de comptage', 'Libelle']).groupby('Libelle')[metatemp_columns].apply(lambda x: chunks(x.values, size=N_input+N_output, step=1))

# print(metatemp_data_train.head())
# print(metatemp_data_train[0])
# print(len(metatemp_data_train.values[0]))

In [8]:
import numpy as np
data_train = df_train.sort_values(['Date et heure de comptage', 'Libelle']).groupby('Libelle')["Taux d'occupation"].apply(lambda x: chunks(x.values, size=N_input+N_output, step=1))

#print(data.values)
data_numpy_train = np.array([np.array(x) for x in data_train.values])
#data_numpy_test = np.array([np.array(x) for x in data_test.values])
metatemp_data_numpy_train = np.array([np.array(x) for x in metatemp_data_train.values])
#metatemp_data_numpy_test = np.array([np.array(x) for x in metatemp_data_test.values])
#data_numpy.astype(float)

print(metatemp_data_numpy_train[0][0][0])

rues = np.array([x for x in data_train.index])


# meta_cat_train = metadata_train[cat_columns].values.astype(int)
# meta_cat_train = np.array([np.array(x) for x in meta_cat_train.values])

# metadata_test = df_meta_test.loc[rues]
# meta_cat_test = metadata_test[cat_columns].values.astype(int)
# meta_cat_test = np.array([np.array(x) for x in meta_cat_test.values])
print(rues)

[  1.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   1.   1.  11.
 143.   0.  79.  11.  16.   9.]
[0 1 2]


In [9]:
# Mélange les rues
print(len(data_numpy_train)==len(metatemp_data_numpy_train))
ind_train = np.arange(len(data_numpy_train))
np.random.shuffle(ind_train)
#ind_test = np.arange(len(data_numpy_test))
#np.random.shuffle(ind_test)
data_numpy = data_numpy_train[ind_train]
#data_numpy_test = data_numpy_test[ind_test]
# meta_cat_train = meta_cat_train[ind_train]
# meta_cat_test = meta_cat_test[ind_test]
metatemp_data_numpy = metatemp_data_numpy_train[ind_train]
#metatemp_data_numpy_test = metatemp_data_numpy_test[ind_test]
print(metatemp_data_numpy_train[0].shape)
print(data_numpy_train[0].shape)

metatemp_data_numpy_train = []
metatemp_data_numpy_test = []
data_numpy_test = []
data_numpy_train = []
#data_numpy_test = data_numpy[nb_train:]
#data_numpy_train = data_numpy[:nb_train]
print(len(data_numpy))
for i in range(len(metatemp_data_numpy)):
    nb_train = int(len(data_numpy[i])*(1-p_test))
    metatemp_data_numpy_train.append(metatemp_data_numpy[i][:nb_train])
    metatemp_data_numpy_test.append(metatemp_data_numpy[i][nb_train:])
    data_numpy_test.append(data_numpy[i][nb_train:])
    data_numpy_train.append(data_numpy[i][:nb_train])
# print(metatemp_data_numpy_train[0].shape)
# print(data_numpy_train[0].shape)
# print(metatemp_data_numpy_train[0][0][0])

True
(9073, 504, 20)
(9073, 504)
3


In [10]:
# Concatène chaque départements ensembles

# meta_cat_train = np.concatenate(meta_cat_train)
# meta_cat_test = np.concatenate(meta_cat_test)
data_numpy_train = np.concatenate(data_numpy_train)
data_numpy_test = np.concatenate(data_numpy_test)
metatemp_data_numpy_train = np.concatenate(metatemp_data_numpy_train)
metatemp_data_numpy_test = np.concatenate(metatemp_data_numpy_test)

# data_numpy_train[data_numpy_train < 0] = 0
# data_numpy_test[data_numpy_test < 0] = 0
# print(metatemp_data_numpy_train.shape)
# print(metatemp_data_numpy_train[0][0][13:])

In [11]:
# Normalise par mean-std calculées sur le data de train
std = data_numpy_train.std((0,1))[None]
mean = data_numpy_train.mean((0,1))[None]
meta_std = metatemp_data_numpy_train[:][:][13:].std((0,1))[None]
meta_mean = metatemp_data_numpy_train[:][:][13:].mean((0,1))[None]
print(meta_std[None][0][0])


data_numpy_train = (data_numpy_train-mean[None])/std[None]
data_numpy_test = (data_numpy_test-mean[None])/std[None]
for i in range (len(metatemp_data_numpy_train)):
    for j in range (len(metatemp_data_numpy_train[i])):
        for k in range (len(num_columns)):
            metatemp_data_numpy_train[i][j][len(cat_columns)+k] = (metatemp_data_numpy_train[i][j][len(cat_columns)+k]-meta_mean[None][0][0][len(cat_columns)+k])/meta_std[None][0][0][len(cat_columns)+k]

for i in range (len(metatemp_data_numpy_test)):
    for j in range (len(metatemp_data_numpy_test[i])):
        for k in range (len(num_columns)):
            metatemp_data_numpy_test[i][j][len(cat_columns)+k] = (metatemp_data_numpy_test[i][j][len(cat_columns)+k]-meta_mean[None][0][0][len(cat_columns)+k])/meta_std[None][0][0][len(cat_columns)+k]
# print(metatemp_data_numpy_train[0][0][13:])
print(metatemp_data_numpy_test.shape)
print(metatemp_data_numpy_train.shape)
def rescale(data):
    return data * std + mean

[5.34586197e-01 6.81377771e-02 3.50686497e-01 3.50686497e-01
 3.50686497e-01 3.48165997e-01 3.50686497e-01 3.48090209e-01
 3.50462225e-01 1.08340305e+00 3.06357718e-01 1.71756402e-01
 4.67703726e-01 6.63027985e+00 7.38648801e+01 3.03395594e-01
 3.41099223e+01 7.02329963e+00 7.61426246e+00 5.74533035e+00]
(0, 504, 20)
(27219, 504, 20)


In [12]:
# Sépare en input/output
X_train, y_train = data_numpy_train[:, :N_input], data_numpy_train[:, N_input:]
X_test, y_test = data_numpy_test[:, :N_input], data_numpy_test[:, N_input:]
#print(metatemp_data_numpy_test)
X_meta_train, y_meta_train = metatemp_data_numpy_train[:, :N_input, :], metatemp_data_numpy_train[:, N_input:, :]
X_meta_test, y_meta_test = metatemp_data_numpy_test[:, :N_input, :], metatemp_data_numpy_test[:, N_input:, :]
X_train = X_train[:,:,None]
y_train = y_train[:,:,None]
X_test = X_test[:,:,None]
y_test = y_test[:,:,None]
print(X_train.shape)
print(X_meta_train.shape)

(27219, 336, 1)
(27219, 336, 20)


In [ ]:
# Install a pip package in the current Jupyter kernel
# import sys
# !{sys.executable} -m pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch

#import torch
device = torch.device('cuda:0')
#print(type(X_train))
#print(data_numpy.dtype)
#print(torch.from_numpy(X_train))
#print(torch.from_numpy(y_train))
#print(torch.from_numpy(y_meta_train))

dataset_train = torch.utils.data.TensorDataset(torch.FloatTensor(X_train),
                                                torch.FloatTensor(X_meta_train),
                                               torch.FloatTensor(y_train),
                                              torch.FloatTensor(y_meta_train))
dataset_test = torch.utils.data.TensorDataset(torch.FloatTensor(X_test),
                                              torch.FloatTensor(X_meta_test),
                                              torch.FloatTensor(y_test),
                                             torch.FloatTensor(y_meta_test))


trainloader = torch.utils.data.DataLoader(
    dataset_train, batch_size=batch_size, shuffle=True, num_workers=4)
testloader = torch.utils.data.DataLoader(
    dataset_test, batch_size=batch_size, shuffle=False, num_workers=4)
for i, data in enumerate(trainloader, 0):
    inputs, metatemp_inputs, target, metatemp_target = data
#     print(inputs)
#     print(metatemp_inputs)

In [ ]:
def fake_baye(outputs, log_sigmas, target):
    loss = 1/2 * (log_sigmas + torch.exp(-log_sigmas) * (outputs - target)**2)
    return loss.mean()

def loss_rescale_factor(log_rescale_factor, outputs, log_sigmas, target):
    batch_size = outputs.shape[0]
    loss = 1/2 * torch.exp(-2 * log_rescale_factor) * torch.sum(torch.exp(-log_sigmas) * (outputs - target)**2)
    loss += batch_size * log_rescale_factor
    return loss.sum()

In [ ]:
import tqdm

def train_model(net, loss_type, optimizer_parameters, epochs=50, gamma=0.001,
                eval_every=1, Lambda=1, alpha=0.5, beta=0.):
    optimizer = torch.optim.Adam(net.parameters(), **optimizer_parameters)
    criterion = torch.nn.MSELoss()

    if hasattr(net, 'log_rescale_factor'):
        optimizer_rescale_factor = torch.optim.Adam([net.log_rescale_factor], **optimizer_parameters)

    t = tqdm.notebook.tqdm(range(epochs))
    for epoch in t:
        for i, data in enumerate(trainloader, 0):

            net.train()
            if hasattr(net, 'log_rescale_factor'):
                net.log_rescale_factor.require_grad = False
            
            
            inputs, metatemp_inputs, target, metatemp_target = data
            inputs = inputs.to(device)
            metatemp_inputs = metatemp_inputs.to(device)
            target = target.to(device)
            metatemp_target = metatemp_target.to(device)
            batch_size, N_output = target.shape[0:2]
            
            #print(inputs)
            #print(metatemp_inputs)
            # forward + backward + optimize
            
            outputs = net(inputs, meta_temporal_input=metatemp_inputs, meta_temporal_target=metatemp_target)
            if len(outputs) == 2:
                outputs, _ = outputs
            else:
                outputs, log_sigmas, _ = outputs

            loss_mse, loss_shape, loss_temporal = torch.tensor(
                0), torch.tensor(0), torch.tensor(0)

            loss = 0
            if 'mse' in loss_type:
                loss_mse = 100 * criterion(outputs, target)
                loss += loss_mse

            if 'dilate' in loss_type:
                loss_dilate, loss_shape, loss_temporal = dilate_loss(
                    outputs, target, alpha, gamma, device)
                loss += loss_dilate

            if 'fake_baye' in loss_type:
                loss_fake_baye = fake_baye(outputs, log_sigmas, target)
                loss += loss_fake_baye

            if 'all_target_regression' in loss_type:
                loss_target_week_regression = F.mse_loss(outputs.sum(1), target.sum(1))
                loss += loss_target_week_regression

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if hasattr(net, 'log_rescale_factor'):
                net.log_rescale_factor.require_grad = True

                loss_rescale = loss_rescale_factor(net.log_rescale_factor, outputs.detach(), log_sigmas.detach(),
                                                   target)
                optimizer_rescale_factor.zero_grad()
                loss_rescale.backward()
                optimizer_rescale_factor.step()

#         if (epoch % eval_every == 0):
            
#             eval_mse, eval_dtw, eval_tdi = eval_model(net, testloader, gamma)

        t.set_postfix(loss=loss.item(),
                      loss_shape=loss_shape.item(),
                      loss_temporal=loss_temporal.item(),
#                       mse=eval_mse,
#                       dtw=eval_dtw,
#                       tdi=eval_tdi
                     )

In [ ]:
def eval_model(net, loader, gamma):
    criterion = torch.nn.MSELoss()
    losses_mse = []
    losses_dtw = []
    losses_tdi = []

    net.eval()

    for i, data in enumerate(loader, 0):
        loss_mse, loss_dtw, loss_tdi = torch.tensor(
            0), torch.tensor(0), torch.tensor(0)
        # get the inputs
        inputs, metatemp_inputs, target, metatemp_target = data
        inputs = inputs.to(device)
        metatemp_inputs = metatemp_inputs.to(device)
        target = target.to(device)
        metatemp_target = metatemp_target.to(device)
        
        batch_size, N_output = target.shape[0:2]
        outputs = net(inputs, meta_temporal_input=metatemp_inputs, meta_temporal_target=metatemp_target)
        if len(outputs) == 2:
            outputs, _ = outputs
        else:
            outputs, prewarp_outputs, warp_matrix = outputs

        # MSE
        loss_mse = criterion(target, outputs)
        loss_dtw, loss_tdi = 0, 0
        # DTW and TDI
        for k in range(batch_size):
            target_k_cpu = target[k, :, 0:1].view(-1).detach().cpu().numpy()
            output_k_cpu = outputs[k, :, 0:1].view(-1).detach().cpu().numpy()

            loss_dtw += dtw(target_k_cpu, output_k_cpu)
            path, sim = dtw_path(target_k_cpu, output_k_cpu)

            Dist = 0
            for i, j in path:
                Dist += (i - j) * (i - j)
            loss_tdi += Dist / (N_output * N_output)

        loss_dtw = loss_dtw / batch_size
        loss_tdi = loss_tdi / batch_size

        # print statistics
        losses_mse.append(loss_mse.item())
        losses_dtw.append(loss_dtw)
        losses_tdi.append(loss_tdi)

    return np.array(losses_mse).mean(), np.array(losses_dtw).mean(), np.array(losses_tdi).mean()


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Encoder(nn.Module):
    def __init__(self, input_dim, enc_hid_dim, dec_hid_dim, num_layers, dropout):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.num_layers = num_layers
      
        self.rnn = nn.GRU(input_dim, enc_hid_dim,
                          num_layers=num_layers,
                          bidirectional=True,
                          batch_first=True,
                          dropout=dropout)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, meta_temporal, hidden=None):
       
        if meta_temporal is not None:
            src = torch.cat([src, meta_temporal], dim=2)
        
        

        outputs, hidden = self.rnn(src, hidden)

        # outputs = [batch size, src len, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]

        hidden = torch.cat((hidden[::2, :, :], hidden[1::2, :, :]), dim=2)
        hidden = self.dropout(hidden)
        hidden = torch.tanh(self.fc(hidden))

        # outputs = [batch size, src len, enc hid dim * 2]
        # hidden = [batch size, dec hid dim]

        return outputs, hidden


class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()

        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):

        # hidden = [batch size, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]

        src_len = encoder_outputs.shape[1]

        # repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)

        # hidden = [batch size, src len, dec hid dim]
        # encoder_outputs = [batch size, src len, enc hid dim * 2]

        energy = torch.tanh(
            self.attn(torch.cat((hidden, encoder_outputs), dim=2)))

        # energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)

        # attention= [batch size, src len]

        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, input_dim, output_dim, enc_hid_dim, dec_hid_dim, fc_dim,
                 num_layers, dropout, attention, meta_temporal_dim, bayesian=False, embed_dim=None):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.num_layers = num_layers
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.fc_dim = fc_dim
        
        self.embed_dim = embed_dim if embed_dim is not None else 0
        self.meta_temporal_dim = meta_temporal_dim if meta_temporal_dim is not None else 0
        
        self.bayesian = bayesian
        
        if self.bayesian:
            self.out_log_sigma = nn.Linear(fc_dim + output_dim, output_dim)

            self.rnn = nn.GRU((enc_hid_dim * 2) + (input_dim * 2) + meta_temporal_dim,
                              dec_hid_dim,
                              num_layers=num_layers,
                              batch_first=True,
                              dropout=dropout)
            self.fc_1 = nn.Linear(
                (enc_hid_dim * 2) + dec_hid_dim + (input_dim * 2) + self.embed_dim, fc_dim)

        else:
            self.rnn = nn.GRU((enc_hid_dim * 2) + output_dim + meta_temporal_dim,
                              dec_hid_dim,
                              num_layers=num_layers,
                              batch_first=True,
                              dropout=dropout)

            self.fc_1 = nn.Linear(
                (enc_hid_dim * 2) + dec_hid_dim + output_dim + self.embed_dim, fc_dim)
        
        self.fc_2 = nn.Linear(fc_dim, fc_dim)
        self.out = nn.Linear(fc_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs, hidden, encoder_outputs, meta_temporal, embeddings=None):

        a = self.attention(hidden[-1, :, :], encoder_outputs)

        # a = [batch size, src len]

        a = a.unsqueeze(1)

        # a = [batch size, 1, src len]

        weighted = torch.bmm(a, encoder_outputs)

        # weighted = [batch size, 1, enc hid dim * 2]

        if meta_temporal is not None:
            rnn_input = torch.cat((inputs, meta_temporal, weighted), dim=2)
        else:
            rnn_input = torch.cat((inputs, weighted), dim=2)

        # rnn_input = [batch size, 1, (enc hid dim * 2) + emb dim]

        output, hidden = self.rnn(rnn_input, hidden)

        # output = [batch size, seq len, dec hid dim * n directions]
        # hidden = [n layers * n directions, batch size, dec hid dim]

        hidden_prediction = torch.cat((output, weighted, inputs), dim=2)
        
        if embeddings is not None:
            hidden_prediction = torch.cat((hidden_prediction, embeddings), dim=2)
        
        #hidden_prediction = self.fc(hidden_prediction)
        hidden_prediction = self.fc_1(F.relu(hidden_prediction))
        hidden_prediction = self.fc_2(F.relu(hidden_prediction))

        prediction = self.out(F.relu(hidden_prediction))

        # prediction = [batch size, output dim]

        if not self.bayesian:
            return prediction, hidden
        else:
            log_sigma = self.out_log_sigma(torch.cat([hidden_prediction, prediction], dim=-1))
            return prediction, log_sigma, hidden


class MetaEmbedder(nn.Module):
    def __init__(self, cat_input_dim, num_input_dim, embed_dim):
        """cat_input_dim should be a tuple (nb_of_inputs, nb_of_categories)"""
        super().__init__()

        self.cat_input_dim = cat_input_dim
        self.num_input_dim = num_input_dim
        self.embed_dim = embed_dim
        # self.cat_module = nn.Embedding(cat_input_dim[1], embed_dim)
        self.num_module = nn.Linear(num_input_dim, embed_dim)
        #self.fc = nn.Linear(self.embed_dim * (cat_input_dim[0] + 1), self.enc_hid_dim * self.num_layers * 2)
        self.fc = nn.Linear(self.embed_dim, self.embed_dim)

    def forward(self, inputs_cat, inputs_num):
        batch_size = inputs_cat.shape[0]
        #x_cat = self.cat_module(inputs_cat).reshape(batch_size, -1)
        x_num = F.relu(self.num_module(inputs_num))
        #x = torch.cat([x_cat, x_num], dim=1)
        x = x_num
        batch_size = x.shape[0]
        out = self.fc(x)
        return out

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device, target_size, embedder=None, mc_dropout=False,
                use_meta_temporal=True):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device
#         print(device)
#         if not torch.cuda.is_available() and self.device is None: 
#             self.device = -1
        
        self.target_size = target_size
      
        self.bayesian = self.decoder.bayesian
        self.mc_dropout = mc_dropout
        self.use_meta_temporal = use_meta_temporal
        
        if embedder is not None:
            self.embedder = embedder
        
        if self.bayesian:
            #Add a learnable rescale factor as in https://openreview.net/pdf?id=CecZ_0t79q
            self.log_rescale_factor = nn.Parameter(torch.ones(1, requires_grad=True))

    def forward(self, src, meta_temporal_input, meta_temporal_target, meta_cat=None, meta_num=None,
               ):
        
        batch_size = src.shape[0]

        if not self.use_meta_temporal:
            meta_temporal_input = None
            meta_temporal_target = None
        
        #print(src)
        #print(meta_temporal_input)
        encoder_outputs, hidden = self.encoder(src, meta_temporal=meta_temporal_input)
        
        embeddings = None
        if hasattr(self, 'embedder') and meta_cat is not None and meta_num is not None:
            embeddings = self.embedder(meta_cat, meta_num).unsqueeze(1)

        # tensor to store decoder outputs
        outputs = torch.zeros(batch_size,
                              self.target_size,
                              self.decoder.output_dim).to(self.device)
        hiddens = torch.zeros(batch_size,
                              self.target_size,
                              self.decoder.dec_hid_dim).to(self.device)
        
        if self.bayesian:
            log_sigmas = torch.zeros(batch_size,
                                     self.target_size,
                                     self.decoder.output_dim).to(self.device)

        inputs = src[:, -1, :].unsqueeze(1)
        
        if self.bayesian:
            inputs = torch.cat([inputs, torch.zeros(*inputs.shape).to(inputs.device)], 2)

        for t in range(0, self.target_size):
            meta_temporal = meta_temporal_target[:, t:t + 1, :] if meta_temporal_target is not None else None
            dec_outputs = self.decoder(inputs, hidden, encoder_outputs, embeddings=embeddings,
                                      meta_temporal=meta_temporal)

            if not self.bayesian:
                output, hidden = dec_outputs
                outputs[:, t:t + 1, :] = output
                hiddens[:, t:t + 1, :] = hidden[-2:-1, :, :].transpose(0, 1)

                inputs = output
            else:
                output, log_sigma, hidden = dec_outputs
                outputs[:, t:t + 1, :] = output
                hiddens[:, t:t + 1, :] = hidden[-2:-1, :, :].transpose(0, 1)
                log_sigmas[:, t:t+1, :] = log_sigma

                inputs = torch.cat([output, log_sigma], dim=2)
        
        #if hasattr(self, 'rescale_factor'):
        #    log_sigmas += 2 * self.rescale_factor

        if not self.bayesian:
            return outputs, hiddens
        else:
            return outputs, log_sigmas, hiddens

def activate_mc_dropout(model):
    for m in model.modules():
        if (type(m) == nn.Dropout) or (type(m) == nn.GRU):
            m.train()

In [ ]:
def gen_net(dropout=0.5, bayesian=False, mc_dropout=False, meta_temporal=True):
    embedder = None
    
    if meta_temporal:
        encoder_input_dim = 1 + len(metatemp_columns)
        meta_temporal_dim = len(metatemp_columns)
    else:
        encoder_input_dim = 1
        meta_temporal_dim = 0
        
    encoder = Encoder(input_dim= encoder_input_dim, enc_hid_dim=32, dec_hid_dim=32,
                      num_layers=4, dropout=dropout
                     ).to(device)
    attention = Attention(enc_hid_dim=32, dec_hid_dim=32)
    decoder = Decoder(input_dim=1, output_dim= 1, enc_hid_dim=32, dec_hid_dim=32, fc_dim=32,
                      num_layers=4, dropout=dropout, attention=attention, bayesian=bayesian,
                      embed_dim = None, meta_temporal_dim=meta_temporal_dim
                     ).to(device)
    net = Seq2Seq(encoder, decoder, device, N_output, mc_dropout=mc_dropout, embedder=embedder, use_meta_temporal=meta_temporal).to(device)

    return net

In [ ]:
#import wandb
print(torch.cuda.device_count())  
print(torch.cuda.is_available())   
print(torch.version.cuda) 
nets_lists = {}

#wandb.init()
#nets_lists['mse_ensembling'] = []
nets_lists['meta_mc_dropout'] = []
for i in range(4):
    net = gen_net(dropout=0.5, bayesian=True, mc_dropout=True, meta_temporal=True)
    train_model(net, loss_type=['fake_baye'], optimizer_parameters=optimizer_parameters,
            epochs=5, eval_every=10)
    nets_lists['meta_mc_dropout'].extend([net]*50)

[x.log_rescale_factor for x in nets_lists['meta_mc_dropout']]

In [ ]:
for i,x in enumerate(nets_lists['meta_mc_dropout']):
    torch.save(x.state_dict(), os.path.join(r"data/model_final_{}.pth").format(i))


In [ ]:
nets_lists = {}
nets_lists['meta_mc_dropout'] = []

#torch.save(x.state_dict(), os.path.join(r"data/model_2_{}.pth").format(i))
#x.eval()
net = gen_net(dropout=0.5, bayesian=True, mc_dropout=True, meta_temporal=True)
net.load_state_dict(torch.load(os.path.join(r"data/modeles_25h/modeles_25h/model_simple_{}.pth").format(0))) 
net.eval()
nets_lists['meta_mc_dropout'].extend([net]*50)
    



In [ ]:
def variational_averaging(preds):
    sigma_shape = np.std(preds, axis=0)
    pred = np.mean(preds, axis=0)
    return pred, sigma_shape, None


def variational_averaging_dba(preds):
    pred = dtw_barycenter_averaging(preds)

    dtw_warps = []
    dtw_paths = []
    for i in range(len(preds)):
        path, dist_i = dtw_path(pred, preds[i])
        w_k = np.zeros((len(pred), len(preds[i])))
        for i, j in path:
            w_k[i, j] = 1.
        dtw_paths.append(path)
        dtw_warps.append(w_k)

    sigma_shape = np.zeros((len(pred), pred.shape[-1]))
    for t in range(len(pred)):
        values = []
        for i in range(len(preds)):
            for j, k in dtw_paths[i]:
                if j == t:
                    values.append(preds[i][k])
        sigma_shape[t] = np.std(values, axis=0)

    dtw_warps = np.array(dtw_warps)
    sigma_time = np.mean(np.abs(np.eye(len(pred)) - dtw_warps))
    sigma_time = np.expand_dims(np.expand_dims(sigma_time, 0), -1)

    return pred, sigma_shape, sigma_time


averaging_fun = {'classical': variational_averaging, 'dba': variational_averaging_dba}

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

test_inputs = torch.FloatTensor(X_test).to(device)
test_targets = torch.FloatTensor(y_test).to(device)
test_metatemp_inputs = torch.FloatTensor(X_meta_test).to(device)
test_metatemp_targets = torch.FloatTensor(y_meta_test).to(device)

sigmas = {n: {k: np.zeros((*test_targets.shape,)) for k in averaging_fun} for n in nets_lists}

predictions = {n: {k: np.zeros((*test_targets.shape,)) for k in averaging_fun} for n in nets_lists}

temp_preds = {n: np.zeros((test_targets.shape[0], len(nets_lists[n]), *test_targets.shape[1:],)) for n in nets_lists}
temp_sigmas = {n: np.zeros((test_targets.shape[0], len(nets_lists[n]), *test_targets.shape[1:],)) for n in nets_lists}



for net_name, nets_list in nets_lists.items():
    with torch.no_grad():
        for net in nets_list:
            net.eval()
            if net.mc_dropout:
                activate_mc_dropout(net)

        for b in range(test_targets.shape[0] // batch_size + int(test_targets.shape[0] % batch_size != 0)):
            preds = []
            aleatoric_sigmas = []
            for net in nets_list:
                pred = net(
                    test_inputs[batch_size * b:batch_size * (b+1), ...],
                    meta_temporal_input=test_metatemp_inputs[batch_size * b:batch_size * (b+1), ...],
                    meta_temporal_target=test_metatemp_targets[batch_size * b:batch_size * (b+1), ...],
                )
                if len(pred) == 2:
                    pred, _ = pred
                else:
                    pred, log_sigma, _ = pred
                    sigma = np.exp(log_sigma.cpu().numpy())
                    sigma = np.squeeze(rescale(sigma))
                    aleatoric_sigmas.append(sigma)

                pred = pred.cpu().numpy()
                pred = rescale(pred)
                preds.append(pred)
            aleatoric_sigmas=np.array(aleatoric_sigmas)[:,:,:,None]
            temp_preds[net_name][batch_size * b:batch_size * (b+1)] = np.array(preds).transpose((1,0,2,3))
            temp_sigmas[net_name][batch_size * b:batch_size * (b+1)] = aleatoric_sigmas.transpose((1,0,2,3))
            
            
            
for ind in tqdm.tqdm_notebook(range(test_targets.shape[0])):
    #print(ind)
    #f = plt.figure(figsize=(24, 12))
    k = 1
    j = 1
    for net_name, nets_list in nets_lists.items():
        #preds = []
        #aleatoric_sigmas = []
#         with torch.no_grad():
#             for net in nets_list:
#                 net.eval()
#                 if net.mc_dropout:
#                     activate_mc_dropout(net)
#                 pred = net(
#                     test_inputs[ind:ind + 1, ...])
#                 if len(pred) == 2:
#                     pred, _ = pred
#                 else:
#                     pred, log_sigma, _ = pred
#                     sigma = np.exp(log_sigma.cpu().numpy()[0, :, :])
#                     sigma = np.squeeze(rescale(sigma))
#                     aleatoric_sigmas.append(sigma)

#                 pred = pred.cpu().numpy()[0, :, :]
#                 pred = rescale(pred)
#                 preds.append(pred)

        #preds = np.array(preds)
        #aleatoric_sigmas = np.array(aleatoric_sigmas)
        preds = temp_preds[net_name][ind]
        aleatoric_sigmas = temp_sigmas[net_name][ind]

        aleatoric_sigmas = np.mean(aleatoric_sigmas, 0) if len(aleatoric_sigmas) != 0 else 0
        if hasattr(nets_list[0], 'log_rescale_factor'):
            aleatoric_sigmas *= np.mean([np.exp(net.log_rescale_factor.cpu().detach().numpy())
                                         for net in nets_list], 0)
        
        if ind % 10 == 0:
            
            f = plt.figure(figsize=(24, 12))
            
        for fun_name, fun in averaging_fun.items():

            pred, sigma_shape, sigma_time = fun(preds)
            
            predictions[net_name][fun_name][ind] = pred

            input = test_inputs.detach().cpu().numpy()[ind, :, :]
            input = rescale(input)
            target = test_targets.detach().cpu().numpy()[ind, :, :]
            target = rescale(target)
            
            epistemic_sigma = 0
            if sigma_shape is not None:
                epistemic_sigma += sigma_shape**2
            if sigma_time is not None:
                epistemic_sigma += sigma_time**2
            epistemic_sigma = np.sqrt(epistemic_sigma)
            
            if hasattr(nets_list[0], 'log_rescale_factor'):
                epistemic_sigma *= np.mean([np.exp(net.log_rescale_factor.cpu().detach().numpy())
                                         for net in nets_list], 0)

            sigma = epistemic_sigma
            sigmas[net_name][fun_name][ind] = sigma
            
            aleatoric_sigmas = np.sqrt(aleatoric_sigmas)

            #plt.subplot(len(nets_lists) + 1, len(averaging_fun) + 1, j + k)
            if ind % 10 == 0:
                plt.subplot(len(nets_lists) + 1, len(averaging_fun) + 1, j + k)
                
                for d in [0]: #range(pred.shape[-1]):
                    plt.plot(range(0, N_input), input[:, d], 'b', label='input', linewidth=1)
                    plt.plot(range(N_input - 1, N_input + N_output),
                             np.concatenate([input[N_input - 1:N_input], target])[:, d],
                             'b--', label='target', linewidth=1)
                    plt.plot(range(N_input - 1, N_input + N_output),
                             np.concatenate([input[N_input - 1:N_input], pred])[:, d],
                             'r', label='prediction', linewidth=1)

                    plt.fill_between(
                        range(N_input - 1, N_input + N_output),
                        np.concatenate(
                            [input[N_input - 1:N_input], pred + 2 * epistemic_sigma]
                        )[:, d],
                        np.concatenate(
                            [input[N_input - 1:N_input], pred - 2 * epistemic_sigma]
                        )[:, d],
                        color='orange', alpha=0.3, label='epistemic', linewidth=1)
            
            if False:
                for d in range(pred.shape[-1]):
                    plt.fill_between(
                        range(N_input - 1, N_input + N_output),
                        np.concatenate(
                            [input[N_input - 1:N_input], pred + 2 * aleatoric_sigmas]
                        )[:, d],
                        np.concatenate(
                            [input[N_input - 1:N_input], pred - 2 * aleatoric_sigmas]
                        )[:, d],
                        color='green', alpha=0.15, label='aleatoric', linewidth=1)
            #plt.xticks(range(0, 28, 2))
            
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys(),
                       loc='upper center', bbox_to_anchor=(0.5, 1.05), ncol=2)

            plt.title('{} {}'.format(net_name, fun_name))
            j = j + 1
        k = k + 1

    plt.show()



In [ ]:
def stat_check(value, samples, method='max'):
    if method == 'max':
        return value > (samples.mean() + 3 * samples.std())
    else:
        return value < (samples.mean() - 3 * samples.std())

def highlight(method='max'):
    def fun(data, bcolor='rgba(60,179,113,0.5)', tcolor='red'):
        '''
        highlight the maximum in a Series or DataFrame
        '''
        m = method if type(method)==str else method[data.name[1]]
        if m == 'min':
            data = -data
        attr = 'background-color: {}'.format(bcolor)

        is_max = data == data.max()

        diff_significative = stat_check(data[is_max].iloc[0], data[~is_max], method=m)
        if diff_significative:
            attr += "; color: {}".format(tcolor)

        return [attr if v else '' for v in is_max]
    return fun

def rower(data):
    s = np.arange(len(data.index)) % 2 != 0
    s = pd.concat([pd.Series(s)] * len(data.columns), axis=1) #6 or the n of cols u have
    z = pd.DataFrame(np.where(s, '', 'background-color:#f2f2f2'),
                 index=data.index, columns=data.columns)
    return z

def set_style(df):
    style = (df.style.apply(rower, axis=None)
             #.apply(highlight({'corr_indiv': 'max','corr_sum':'max','mae':'min','rmse':'min','smape':'min'}))
             .set_properties(**{'text-align': 'right', 'border': '1px solid black'})
             .set_table_styles([{'selector': 'th', 'props': {'text-align': 'right', 'border': '2px solid black'}.items()}])
            )
    return style


In [ ]:
import sklearn.metrics as metrics

def filter_nans(a, b):
    cond = np.isnan(a).any(1) | (a == np.inf).any(1) | (a == -np.inf).any(1)
    return a[~cond], b[~cond]

def smape(pred, target):
    pred, target = filter_nans(pred, target)
    smape = 100 * np.mean(np.abs(target - pred) / (np.abs(pred) + np.abs(target)), (0,1))
    if smape > 100:
        smape = 100.
    return smape

def mae(pred, target):
    pred, target = filter_nans(pred, target)
    return metrics.mean_absolute_error(pred, target)


def rmse(pred, target):
    pred, target = filter_nans(pred, target)
    return np.mean(np.sqrt(np.sum((pred - target)**2, 1)), 0)

def corr_indiv(pred, target):
    pred, target = filter_nans(pred, target)
    return np.corrcoef(pred.reshape(-1), target.reshape(-1))[0, 1]

def corr_sum(pred, target):
    pred, target = filter_nans(pred, target)
    return np.corrcoef(pred.sum(1), target.sum(1))[0, 1]

In [ ]:
list_metrics = {'smape': smape, 'mae': mae, 'rmse': rmse, 'corr_indiv': corr_indiv, 'corr_sum': corr_sum}

In [ ]:
def compute_results(predictions, models, averaging_fun=['classical']):
    resultats =  {l: {n: {} for n in averaging_fun} for l in list_metrics}
    for i, net_name in enumerate(models):
        for fun_name in predictions[net_name]:
            print(predictions[net_name][fun_name].shape)
            print(rescale(y_test)[-1,-15:-1,0])
            for metric_name, fun in list_metrics.items(): 
                m = fun(predictions[net_name][fun_name][:,:,0],rescale(y_test)[:,:,0])
                resultats[metric_name][fun_name][net_name] = m
                
    resultats = pd.DataFrame.from_dict({(j,k): resultats[j][k]
                                        for j in resultats.keys()
                                        for k in resultats[j].keys()},
                                       orient='columns')
    print(resultats)
    resultats = resultats.stack()
    print(resultats)
    resultats.rename_axis(['Méthode', 'Aggrégation'], inplace=True)
    resultats.rename_axis(['Metric'], axis=1, inplace=True)
    return resultats

In [ ]:
resultats_dl = compute_results(predictions, nets_lists, averaging_fun=averaging_fun)
set_style(resultats_dl)

In [ ]:
print(resultats_dl)